<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/275_MissionOrchestratorAgent_Toolshed_KPIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KPI metric calculation utilities

In [ ]:
"""KPI metric calculation utilities"""

from typing import List, Dict, Any


def calculate_kpi_metrics(
    executed_tasks: List[Dict[str, Any]],
    kpi_definitions: Dict[str, Any]
) -> Dict[str, Any]:
    """
    Calculate current KPI metrics based on executed tasks.

    This is a generic implementation that works with any KPI structure.
    It calculates:
    - Total duration (in minutes, hours, days)
    - Number of completed items
    - Improvement percentages (if baselines provided)

    Args:
        executed_tasks: List of completed task execution results
        kpi_definitions: KPI definitions dictionary (mission-specific structure)

    Returns:
        Dictionary of calculated KPI metrics
    """
    metrics = {}

    # Calculate total time (sum of all task durations)
    total_duration_minutes = sum(
        task.get("duration_minutes", 0) for task in executed_tasks
        if task.get("status") == "completed"
    )

    # Convert to different time units
    total_duration_days = total_duration_minutes / (60 * 24)
    total_duration_hours = total_duration_minutes / 60.0

    # Calculate number of steps (completed tasks)
    steps_completed = len([
        task for task in executed_tasks
        if task.get("status") == "completed"
    ])

    # Map to KPI definitions (generic approach)
    # This handles common KPI patterns:
    # - target_*_time_days -> actual_*_time_days
    # - target_*_time_hours -> actual_*_time_hours
    # - max_steps -> actual_steps
    # - min_touchpoints -> actual_touchpoints

    for key in kpi_definitions.keys():
        if key.startswith("target_") and key.endswith("_days"):
            metric_name = key.replace("target_", "actual_")
            metrics[metric_name] = round(total_duration_days, 2)

        elif key.startswith("target_") and key.endswith("_hours"):
            metric_name = key.replace("target_", "actual_")
            metrics[metric_name] = round(total_duration_hours, 2)

        elif key == "max_steps":
            metrics["actual_steps"] = steps_completed

        elif key == "min_touchpoints":
            metrics["actual_touchpoints"] = steps_completed

    # Calculate improvement percentage if baseline exists
    # Look for baseline_* keys and calculate improvement
    for key in kpi_definitions.keys():
        if key.startswith("baseline_"):
            baseline = kpi_definitions[key]

            # Find corresponding actual metric
            # baseline_onboarding_time_days -> actual_onboarding_time_days
            actual_key = key.replace("baseline_", "actual_")
            actual_value = metrics.get(actual_key, 0)

            if baseline > 0 and actual_value is not None:
                improvement = ((baseline - actual_value) / baseline) * 100
                metrics["improvement_percentage"] = round(improvement, 2)
                break  # Only calculate one improvement percentage

    return metrics




# ✅ KPI Metric Calculation Utility — Summary

This utility takes all the **completed task results** and uses them to compute the **actual KPI values** the mission cares about.

Think of it like the agent saying:

> “Let me total up my performance so far.”

It doesn’t judge the performance — it just **calculates the numbers**, like a scoreboard.
The *next* utility interprets those numbers.

This function produces values such as:

* actual time spent
* actual steps taken
* actual touchpoints
* improvement percentage compared to baseline

These become inputs to:

* progress tracking
* KPI assessment
* reporting
* mission summaries

---

# ⭐ What It Calculates

The function computes the following:

---

## **1️⃣ Total duration across all completed tasks**

It sums all `duration_minutes` fields:

* total minutes
* total hours
* total days

These values become the basis for time-based KPIs like:

* `actual_onboarding_time_days`
* `actual_pipeline_days`
* `actual_resolution_time_hours`

---

## **2️⃣ Total number of completed steps**

It counts every task with:

```python
status == "completed"
```

This value is used for KPIs like:

* `actual_steps` (when `max_steps` is defined)
* `actual_touchpoints` (when `min_touchpoints` is defined)

In many business workflows, “steps completed” or “touchpoints” is a key measure of efficiency.

---

## **3️⃣ Automatic mapping to KPI definitions**

This part is especially clever.

Instead of hardcoding KPIs, the function does pattern matching:

* if a key is `target_*_days` → produce `actual_*_days`
* if a key is `target_*_hours` → produce `actual_*_hours`
* if KPI is `max_steps` → produce `actual_steps`
* if KPI is `min_touchpoints` → produce `actual_touchpoints`

This means:

> You can define ANY KPI structure in your mission file,
> and this utility will adapt automatically.

This is a huge win for flexibility.

---

## **4️⃣ Improvement percentage**

If a baseline exists (like baseline times or baseline steps), it computes:

```python
((baseline - actual) / baseline) * 100
```

This tells you:

> “How much better did we perform compared to the old way of doing things?”

It stops after the first baseline match, which keeps things simple.

---

# 🎯 Big Picture

This utility turns **raw execution data** into **meaningful KPI metrics**:

* It is *generic*, so every mission can define its own KPIs
* It is *automatic*, so you don’t need custom logic for each orchestrator
* It is *simple*, requiring only task durations + definitions
* It is *compatible* with the KPI assessment utility you summarized earlier
* It is *report-ready*, giving clean numbers for your mission reports

It is essentially the **math engine** of your KPI subsystem.

You now have:

* KPI metric calculation (this function)
* KPI status interpretation (previous utility)

Together, they form a complete, reusable KPI module in your toolshed.




# ROI calculation utilities

In [ ]:
"""ROI calculation utilities"""


def calculate_roi_improvement(baseline: float, actual: float) -> float:
    """
    Calculate ROI improvement percentage.

    Args:
        baseline: Baseline value
        actual: Actual value

    Returns:
        Improvement percentage (positive = improvement, negative = degradation)
    """
    if baseline == 0:
        return 0.0

    improvement = ((baseline - actual) / baseline) * 100.0
    return round(improvement, 2)




# 📈 ROI Improvement Utility — Summary

This is a tiny function, but it's surprisingly powerful because it gives your agent the ability to measure **how much better (or worse)** the new workflow performed compared to the old one.

At its core it answers one question:

> **“Did we improve compared to the baseline?”**

---

# ⭐ What It Does

It takes two values:

* **baseline** → how things *used* to be
* **actual** → how things turned out this time

And it returns a simple percentage showing improvement.

---

## ✔️ How it works

It uses the universal ROI formula:

```
(baseline - actual) / baseline * 100
```

If actual < baseline → **positive improvement**
If actual > baseline → **negative improvement** (you got worse)

Example:

* Baseline time: 10 days
* Actual time: 5 days

ROI = 50% improvement 🎉

---

# ⭐ Why it’s useful

Even though the function is small, it's valuable because:

* It gives **simple, clear performance signals**
* It works for any numeric KPI (time, steps, cost, effort, etc.)
* It can be reused across missions, agents, and orchestrators
* It gives stakeholders a metric they instantly understand

It also plugs neatly into:

* KPI metrics
* KPI status
* Reporting
* ROI dashboards
* Mission performance summaries

---

# 🎯 Takeaway

This utility is like the **magnesium spark** in a chemistry set:

Small, lightweight, but it triggers the “Wow, we improved!” moment.

It's a perfect example of why reusable tools in your toolshed are so powerful—even tiny ones can dramatically increase clarity in your agent ecosystem.



# KPI status assessment utilities

In [ ]:
"""KPI status assessment utilities"""

from typing import Dict, Any


def assess_kpi_status(
    kpi_metrics: Dict[str, Any],
    kpi_definitions: Dict[str, Any],
    warning_threshold: float = 0.8,
    critical_threshold: float = 0.5
) -> Dict[str, str]:
    """
    Assess KPI status (on_track, at_risk, exceeded).

    Generic implementation that works with any KPI structure.

    Args:
        kpi_metrics: Calculated KPI metrics
        kpi_definitions: KPI definitions with targets
        warning_threshold: Warning threshold (0.8 = 80% of target)
        critical_threshold: Critical threshold (0.5 = 50% of target)

    Returns:
        Dictionary mapping KPI names to status strings
    """
    status = {}

    # Generic assessment for time-based KPIs (target_*_time_days, target_*_time_hours)
    for target_key in kpi_definitions.keys():
        if target_key.startswith("target_") and (target_key.endswith("_days") or target_key.endswith("_hours")):
            target = kpi_definitions[target_key]
            actual_key = target_key.replace("target_", "actual_")
            actual = kpi_metrics.get(actual_key)

            if actual is not None:
                kpi_name = target_key.replace("target_", "").replace("_days", "").replace("_hours", "")

                if actual <= target:
                    status[kpi_name] = "exceeded"  # Better than target
                elif actual <= target / critical_threshold:
                    status[kpi_name] = "on_track"
                elif actual <= target / warning_threshold:
                    status[kpi_name] = "at_risk"
                else:
                    status[kpi_name] = "at_risk"

        # Generic assessment for max_* KPIs (e.g., max_steps)
        elif target_key.startswith("max_"):
            target = kpi_definitions[target_key]
            metric_key = target_key.replace("max_", "actual_")
            actual = kpi_metrics.get(metric_key)

            if actual is not None:
                kpi_name = target_key.replace("max_", "")

                if actual <= target:
                    status[kpi_name] = "on_track"
                else:
                    status[kpi_name] = "at_risk"

        # Generic assessment for min_* KPIs (e.g., min_touchpoints)
        elif target_key.startswith("min_"):
            target = kpi_definitions[target_key]
            metric_key = target_key.replace("min_", "actual_")
            actual = kpi_metrics.get(metric_key)

            if actual is not None:
                kpi_name = target_key.replace("min_", "")

                if actual >= target:
                    status[kpi_name] = "on_track"
                else:
                    status[kpi_name] = "at_risk"

    return status





# ✅ KPI Status Assessment Utility — Summary

This utility takes the **raw KPI measurements** your orchestrator has calculated (e.g., actual time, actual steps, actual touchpoints) and compares them to your **KPI goals** (targets, minimums, maximums).
Its job is to decide:

> “Is this KPI looking good, okay, or risky?”

It returns labels like:

* `"exceeded"` → doing better than the goal
* `"on_track"` → meeting expectations
* `"at_risk"` → falling behind or not hitting the metric

This gives the orchestrator a **quick health check** on performance.

---

# ⭐ What It Checks

This function supports three common KPI types:

---

## **1️⃣ Time-based KPIs: `target_*_days` or `target_*_hours`**

Example:

* `target_pipeline_days`
* `target_resolution_time_hours`

The rules:

* If the agent finishes faster than the target → `"exceeded"`
* If the actual time is within reasonable range → `"on_track"`
* If the actual time is drifting higher → `"at_risk"`

The warning and critical thresholds (80% and 50%) help classify how close we are to slipping behind.

In simple terms:

> “The quicker the agent finishes, the better the status.”

---

## **2️⃣ Maximum KPIs: `max_*`**

Example:

* `max_steps`
* `max_touchpoints`

The rule is simple:

* If actual ≤ max → `"on_track"`
* If actual > max → `"at_risk"`

This is used for things where **less is better**.

---

## **3️⃣ Minimum KPIs: `min_*`**

Example:

* `min_touchpoints`
* `min_engagement_score`

Rule:

* If actual ≥ min → `"on_track"`
* Otherwise → `"at_risk"`

Used when **more is better**.

---

# ⭐ Why This Utility Matters

This function is important because it allows your orchestrator to:

* summarize mission performance
* detect problems early
* feed clear performance signals to humans
* include KPI status in the final report
* adjust routing in more advanced frameworks (future)

It turns raw numbers into **understandable judgments**.

Instead of:

> “Actual pipeline days = 3.2, target = 5”

You get:

> `"pipeline": "exceeded"`

Which is much easier to read, visualize, and react to.

---

# 🎯 The Big Picture

You now have:

* utilities that **calculate KPIs** (actual values)
* utilities that **compute improvement vs baseline**
* **this utility**, which converts numbers into easy-to-understand labels

Together, these form a **complete KPI module** that:

* informs the report
* informs the progress tracker
* gives humans instant performance insight

This is exactly the kind of reusable logic you *want* in the toolshed —
every orchestrator benefits from clean, consistent KPI assessments.

